# Final Model Comparison: All Models, All Balancing Schemes

This notebook compares all machine learning models trained for attrition prediction:
- **Logistic Regression**
- **Decision Tree**
- **Random Forest**
- **Gradient Boosting**
- **SVC (RBF Kernel)**
- **MLP (Neural Network)**

Each model is evaluated on three datasets:
- **Raw / Original** (Imbalanced)
- **SMOTE-Balanced** (Synthetic Oversampling)
- **ADASYN-Balanced** (Adaptive Synthetic Oversampling)

## Metrics
All results are evaluated on the same test set(s) using:
- Accuracy
- Precision (positive class)
- Recall (positive class, focus metric)
- F1 (positive class)
- ROC AUC
- Confusion Matrix (TN, FP, FN, TP)

**Objective:**
- Aggregate all scoring results, highlight the best overall model/dataset per metric, and deliver business-facing recommendations.
---



## Step 1: Aggregation and Loading of All Model Results

> ⚠️ **Instruction:** If you have stored metrics as pickles or CSVs from each modeling notebook, load them below. Otherwise, manually copy the `all_metrics` lists / DataFrames (with all relevant fields) from the end of each notebook.

**Expected unified DataFrame columns:**
- `model_name`, `dataset_type`, `accuracy`, `precision_positive`, `recall_positive`, `f1_positive`, `roc_auc`, `tn`, `fp`, `fn`, `tp`

This cell loads and combines results from all notebooks.


In [19]:
# ============================================================
# Automatically Extract and Combine Metrics from All Notebooks
# ============================================================
import pandas as pd
import numpy as np
import json
import os
import re

def extract_metrics_from_notebook03():
    """Extract metrics from notebook 03 (tree-based models)."""
    notebook_path = '../03-tree-based-models.ipynb'
    if not os.path.exists(notebook_path):
        return []
    
    try:
        with open(notebook_path, 'r') as f:
            nb = json.load(f)
        
        metrics_list = []
        cm_data = {}  # Store confusion matrix data by model-dataset
        
        # First pass: Extract performance metrics
        for cell in nb['cells']:
            if cell['cell_type'] == 'code' and 'outputs' in cell:
                for output in cell['outputs']:
                    if 'text' in output:
                        text = ''.join(output['text'])
                        
                        # Extract performance metrics
                        if 'PERFORMANCE METRICS' in text or 'COMPREHENSIVE METRICS' in text:
                            lines = text.split('\n')
                            for line in lines:
                                if any(m in line for m in ['DecisionTree', 'RandomForest', 'GradientBoosting']):
                                    # Parse line like: "    DecisionTree    Raw/Original  0.727891   0.326241 0.647887 0.433962 0.692977"
                                    parts = line.split()
                                    if len(parts) >= 7:
                                        try:
                                            # Find model name (first non-empty word)
                                            model_idx = 0
                                            while model_idx < len(parts) and not parts[model_idx]:
                                                model_idx += 1
                                            model = parts[model_idx] if model_idx < len(parts) else None
                                            
                                            # Find dataset (next 1-2 words)
                                            dataset_start = model_idx + 1
                                            if dataset_start < len(parts):
                                                if dataset_start + 1 < len(parts) and parts[dataset_start + 1] in ['Balanced', 'Original']:
                                                    dataset_str = ' '.join(parts[dataset_start:dataset_start+2])
                                                    num_start = dataset_start + 2
                                                else:
                                                    dataset_str = parts[dataset_start]
                                                    num_start = dataset_start + 1
                                                
                                                dataset_type = 'raw' if 'Raw' in dataset_str or 'Original' in dataset_str else ('smote' if 'SMOTE' in dataset_str else 'adasyn')
                                                
                                                # Extract numeric values
                                                nums = []
                                                for p in parts[num_start:]:
                                                    try:
                                                        nums.append(float(p))
                                                    except:
                                                        pass
                                                
                                                if len(nums) >= 5 and model:
                                                    metrics_list.append({
                                                        'model_name': model,
                                                        'dataset_type': dataset_type,
                                                        'accuracy': nums[0],
                                                        'precision_positive': nums[1],
                                                        'recall_positive': nums[2],
                                                        'f1_positive': nums[3],
                                                        'roc_auc': nums[4],
                                                        'tn': 0, 'fp': 0, 'fn': 0, 'tp': 0
                                                    })
                                        except Exception as e:
                                            pass
                        
                        # Extract confusion matrix components
                        if 'CONFUSION MATRIX COMPONENTS' in text or 'Confusion Matrix Components' in text:
                            lines = text.split('\n')
                            for line in lines:
                                if any(m in line for m in ['DecisionTree', 'RandomForest', 'GradientBoosting']):
                                    # Parse line like: "    DecisionTree    Raw/Original 275  95  25  46"
                                    parts = line.split()
                                    if len(parts) >= 7:
                                        try:
                                            # Find model name
                                            model_idx = 0
                                            while model_idx < len(parts) and not parts[model_idx]:
                                                model_idx += 1
                                            model = parts[model_idx] if model_idx < len(parts) else None
                                            
                                            # Find dataset
                                            dataset_start = model_idx + 1
                                            if dataset_start < len(parts):
                                                if dataset_start + 1 < len(parts) and parts[dataset_start + 1] in ['Balanced', 'Original']:
                                                    dataset_str = ' '.join(parts[dataset_start:dataset_start+2])
                                                    num_start = dataset_start + 2
                                                else:
                                                    dataset_str = parts[dataset_start]
                                                    num_start = dataset_start + 1
                                                
                                                dataset_type = 'raw' if 'Raw' in dataset_str or 'Original' in dataset_str else ('smote' if 'SMOTE' in dataset_str else 'adasyn')
                                                
                                                # Get integers for confusion matrix
                                                nums = [int(p) for p in parts[num_start:] if p.isdigit()]
                                                if len(nums) >= 4 and model:
                                                    key = f"{model}_{dataset_type}"
                                                    cm_data[key] = {'tn': nums[0], 'fp': nums[1], 'fn': nums[2], 'tp': nums[3]}
                                        except:
                                            pass
        
        # Merge confusion matrix data into metrics
        for metric in metrics_list:
            key = f"{metric['model_name']}_{metric['dataset_type']}"
            if key in cm_data:
                metric.update(cm_data[key])
        
        return metrics_list
    except Exception as e:
        print(f"Error extracting from notebook 03: {e}")
        return []

def extract_metrics_from_notebook04():
    """Extract metrics from notebook 04 (SVC and MLP)."""
    notebook_path = '../04-svc-nn-models.ipynb'
    if not os.path.exists(notebook_path):
        return []
    
    try:
        with open(notebook_path, 'r') as f:
            nb = json.load(f)
        
        metrics_list = []
        cm_data = {}  # Store confusion matrix data by model-dataset
        
        # First pass: Extract performance metrics
        for cell in nb['cells']:
            if cell['cell_type'] == 'code' and 'outputs' in cell:
                for output in cell['outputs']:
                    if 'text' in output:
                        text = ''.join(output['text'])
                        
                        # Extract performance metrics
                        if 'Performance Metrics' in text or 'COMPREHENSIVE METRICS' in text:
                            lines = text.split('\n')
                            for line in lines:
                                if 'SVC' in line or 'MLP' in line:
                                    # Parse line like: "       SVC    Raw/Original  0.857143            0.560976         0.489362     0.522727 0.778620"
                                    parts = line.split()
                                    if len(parts) >= 7:
                                        try:
                                            # Find model name (first non-empty word that's SVC or MLP)
                                            model_idx = 0
                                            while model_idx < len(parts) and parts[model_idx] not in ['SVC', 'MLP']:
                                                model_idx += 1
                                            model = parts[model_idx] if model_idx < len(parts) else None
                                            
                                            # Find dataset (next 1-2 words)
                                            dataset_start = model_idx + 1
                                            if dataset_start < len(parts):
                                                if dataset_start + 1 < len(parts) and parts[dataset_start + 1] in ['Balanced', 'Original']:
                                                    dataset_str = ' '.join(parts[dataset_start:dataset_start+2])
                                                    num_start = dataset_start + 2
                                                else:
                                                    dataset_str = parts[dataset_start]
                                                    num_start = dataset_start + 1
                                                
                                                dataset_type = 'raw' if 'Raw' in dataset_str or 'Original' in dataset_str else ('smote' if 'SMOTE' in dataset_str else 'adasyn')
                                                
                                                # Extract numeric values
                                                nums = []
                                                for p in parts[num_start:]:
                                                    try:
                                                        nums.append(float(p))
                                                    except:
                                                        pass
                                                
                                                if len(nums) >= 5 and model:
                                                    metrics_list.append({
                                                        'model_name': model,
                                                        'dataset_type': dataset_type,
                                                        'accuracy': nums[0],
                                                        'precision_positive': nums[1],
                                                        'recall_positive': nums[2],
                                                        'f1_positive': nums[3],
                                                        'roc_auc': nums[4],
                                                        'tn': 0, 'fp': 0, 'fn': 0, 'tp': 0
                                                    })
                                        except Exception as e:
                                            pass
                        
                        # Extract confusion matrix components
                        if 'Confusion Matrix Components' in text:
                            lines = text.split('\n')
                            for line in lines:
                                if 'SVC' in line or 'MLP' in line:
                                    # Parse line like: "       SVC    Raw/Original 229  18  24  23"
                                    parts = line.split()
                                    if len(parts) >= 7:
                                        try:
                                            # Find model name
                                            model_idx = 0
                                            while model_idx < len(parts) and parts[model_idx] not in ['SVC', 'MLP']:
                                                model_idx += 1
                                            model = parts[model_idx] if model_idx < len(parts) else None
                                            
                                            # Find dataset
                                            dataset_start = model_idx + 1
                                            if dataset_start < len(parts):
                                                if dataset_start + 1 < len(parts) and parts[dataset_start + 1] in ['Balanced', 'Original']:
                                                    dataset_str = ' '.join(parts[dataset_start:dataset_start+2])
                                                    num_start = dataset_start + 2
                                                else:
                                                    dataset_str = parts[dataset_start]
                                                    num_start = dataset_start + 1
                                                
                                                dataset_type = 'raw' if 'Raw' in dataset_str or 'Original' in dataset_str else ('smote' if 'SMOTE' in dataset_str else 'adasyn')
                                                
                                                # Get integers for confusion matrix
                                                nums = [int(p) for p in parts[num_start:] if p.isdigit()]
                                                if len(nums) >= 4 and model:
                                                    key = f"{model}_{dataset_type}"
                                                    cm_data[key] = {'tn': nums[0], 'fp': nums[1], 'fn': nums[2], 'tp': nums[3]}
                                        except:
                                            pass
        
        # Merge confusion matrix data into metrics
        for metric in metrics_list:
            key = f"{metric['model_name']}_{metric['dataset_type']}"
            if key in cm_data:
                metric.update(cm_data[key])
        
        return metrics_list
    except Exception as e:
        print(f"Error extracting from notebook 04: {e}")
        return []

# Extract metrics from all notebooks
print("=" * 70)
print("EXTRACTING METRICS FROM ALL NOTEBOOKS")
print("=" * 70)

all_combined_metrics = []

# Extract from notebook 03
print("\nExtracting from notebook 03 (Tree-based models)...")
metrics_tree = extract_metrics_from_notebook03()
print(f"  Found {len(metrics_tree)} entries from notebook 03")
if len(metrics_tree) > 0:
    all_combined_metrics.extend(metrics_tree)
    print(f"  Models found: {set(m['model_name'] for m in metrics_tree)}")

# Extract from notebook 04
print("\nExtracting from notebook 04 (SVC and MLP)...")
metrics_svc_mlp = extract_metrics_from_notebook04()
print(f"  Found {len(metrics_svc_mlp)} entries from notebook 04")
if len(metrics_svc_mlp) > 0:
    all_combined_metrics.extend(metrics_svc_mlp)
    print(f"  Models found: {set(m['model_name'] for m in metrics_svc_mlp)}")

# Note: Notebook 02 (Logistic Regression) uses a different format
# Manual entry section below
print("\n" + "=" * 70)
print("NOTEBOOK 02 (Logistic Regression) - Manual Entry Required")
print("=" * 70)
print("After running notebook 02, extract metrics from model_results['comparison_df']")
print("and model_results['raw/smote/adasyn']['confusion_matrix']")
print("\nTemplate (uncomment and fill after running notebook 02):")
print("# metrics_logreg = [")
print("#     {'model_name': 'LogisticRegression', 'dataset_type': 'raw', ...},")
print("#     {'model_name': 'LogisticRegression', 'dataset_type': 'smote', ...},")
print("#     {'model_name': 'LogisticRegression', 'dataset_type': 'adasyn', ...}")
print("# ]")
print("# all_combined_metrics.extend(metrics_logreg)")

# Create DataFrame
if len(all_combined_metrics) > 0:
    metrics_df = pd.DataFrame(all_combined_metrics)
    print(f"\n{'='*70}")
    print(f"SUCCESS: Combined {len(metrics_df)} model-dataset combinations")
    print(f"Models: {sorted(metrics_df['model_name'].unique().tolist())}")
    print(f"Datasets: {sorted(metrics_df['dataset_type'].unique().tolist())}")
    print(f"{'='*70}")
else:
    metrics_df = pd.DataFrame(columns=[
        'model_name', 'dataset_type',
        'accuracy', 'precision_positive', 'recall_positive', 'f1_positive', 'roc_auc',
        'tn', 'fp', 'fn', 'tp'
    ])
    print(f"\n{'='*70}")
    print("WARNING: No metrics extracted.")
    print("Please ensure notebooks 03 and 04 have been executed completely.")
    print(f"{'='*70}")

metrics_df


EXTRACTING METRICS FROM ALL NOTEBOOKS

Extracting from notebook 03 (Tree-based models)...
  Found 0 entries from notebook 03

Extracting from notebook 04 (SVC and MLP)...
  Found 0 entries from notebook 04

NOTEBOOK 02 (Logistic Regression) - Manual Entry Required
After running notebook 02, extract metrics from model_results['comparison_df']
and model_results['raw/smote/adasyn']['confusion_matrix']

Template (uncomment and fill after running notebook 02):
# metrics_logreg = [
#     {'model_name': 'LogisticRegression', 'dataset_type': 'raw', ...},
#     {'model_name': 'LogisticRegression', 'dataset_type': 'smote', ...},
#     {'model_name': 'LogisticRegression', 'dataset_type': 'adasyn', ...}
# ]
# all_combined_metrics.extend(metrics_logreg)

Please ensure notebooks 03 and 04 have been executed completely.


,model_name,dataset_type,accuracy,precision_positive,recall_positive,f1_positive,roc_auc,tn,fp,fn,tp


## Step 1b: Manual Entry for Notebook 02 (Logistic Regression)

If automatic extraction didn't find notebook 02 metrics, manually add them here after running notebook 02.


In [20]:
# ============================================================
# Manual Entry for Notebook 02: Logistic Regression
# ============================================================
# After running notebook 02, copy the metrics here from model_results

# Example format (uncomment and fill with actual values from notebook 02):
# metrics_logreg = [
#     {
#         'model_name': 'LogisticRegression',
#         'dataset_type': 'raw',
#         'accuracy': 0.8934,  # From model_results['comparison_df'] for Raw/Original
#         'precision_positive': 0.8158,
#         'recall_positive': 0.4366,
#         'f1_positive': 0.5688,
#         'roc_auc': 0.8165,
#         'tn': 0,  # Extract from model_results['raw']['confusion_matrix']
#         'fp': 0,
#         'fn': 0,
#         'tp': 0
#     },
#     {
#         'model_name': 'LogisticRegression',
#         'dataset_type': 'smote',
#         'accuracy': 0.8073,
#         'precision_positive': 0.4286,
#         'recall_positive': 0.5915,
#         'f1_positive': 0.4970,
#         'roc_auc': 0.7977,
#         'tn': 0,
#         'fp': 0,
#         'fn': 0,
#         'tp': 0
#     },
#     {
#         'model_name': 'LogisticRegression',
#         'dataset_type': 'adasyn',
#         'accuracy': 0.8141,
#         'precision_positive': 0.4421,
#         'recall_positive': 0.5915,
#         'f1_positive': 0.5060,
#         'roc_auc': 0.8002,
#         'tn': 0,
#         'fp': 0,
#         'fn': 0,
#         'tp': 0
#     }
# ]

# Uncomment to add to combined metrics:
# if 'metrics_logreg' in locals():
#     all_combined_metrics.extend(metrics_logreg)
#     metrics_df = pd.DataFrame(all_combined_metrics)
#     print(f"Added {len(metrics_logreg)} Logistic Regression entries. Total: {len(metrics_df)}")

print("Uncomment and fill in metrics_logreg above after running notebook 02")


Uncomment and fill in metrics_logreg above after running notebook 02


In [21]:
# --- Defensive dataset name mapping ---
if len(metrics_df) > 0:
    if 'dataset' not in metrics_df.columns and 'dataset_type' in metrics_df.columns:
        dataset_name_map = {'raw': 'Raw/Original', 'smote': 'SMOTE Balanced', 'adasyn': 'ADASYN Balanced'}
        metrics_df['dataset'] = metrics_df['dataset_type'].map(dataset_name_map)
    
    print(f"\nFinal metrics DataFrame shape: {metrics_df.shape}")
    print(f"Columns: {list(metrics_df.columns)}")
else:
    print("\nMetrics DataFrame is empty. Please run notebooks 03 and 04, and optionally add notebook 02 metrics manually.")




Metrics DataFrame is empty. Please run notebooks 03 and 04, and optionally add notebook 02 metrics manually.


## Step 1c: Verify Metrics Extraction

Check what metrics were successfully extracted and what might be missing.


In [26]:
# ============================================================
# Verify Metrics Extraction
# ============================================================

if len(metrics_df) > 0:
    print("=" * 70)
    print("METRICS EXTRACTION SUMMARY")
    print("=" * 70)
    
    print(f"\nTotal model-dataset combinations: {len(metrics_df)}")
    print(f"\nModels found: {sorted(metrics_df['model_name'].unique().tolist())}")
    print(f"Datasets found: {sorted(metrics_df['dataset_type'].unique().tolist())}")
    
    # Check for missing models
    expected_models = ['LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'SVC', 'MLP']
    found_models = set(metrics_df['model_name'].unique())
    missing_models = set(expected_models) - found_models
    
    if missing_models:
        print(f"\n⚠️  Missing models: {sorted(missing_models)}")
        print("   - Run the corresponding notebooks and re-run extraction")
    else:
        print("\n✓ All expected models found!")
    
    # Check for missing datasets
    expected_datasets = ['raw', 'smote', 'adasyn']
    found_datasets = set(metrics_df['dataset_type'].unique())
    missing_datasets = set(expected_datasets) - found_datasets
    
    if missing_datasets:
        print(f"\n⚠️  Missing datasets: {sorted(missing_datasets)}")
    else:
        print("\n✓ All expected datasets found!")
    
    # Check for missing confusion matrix data
    missing_cm = metrics_df[(metrics_df['tn'] == 0) & (metrics_df['fp'] == 0) & 
                            (metrics_df['fn'] == 0) & (metrics_df['tp'] == 0)]
    if len(missing_cm) > 0:
        print(f"\n⚠️  {len(missing_cm)} entries missing confusion matrix data")
        print("   These will need to be filled manually or re-extracted")
    else:
        print("\n✓ All confusion matrix data present!")
    
    print("\n" + "=" * 70)
    print("Ready for comparison analysis!")
    print("=" * 70)
else:
    print("=" * 70)
    print("NO METRICS FOUND")
    print("=" * 70)
    print("\nPlease:")
    print("1. Ensure notebooks 03 and 04 have been executed completely")
    print("2. Re-run the extraction cell above")
    print("3. Optionally add notebook 02 metrics manually")
    print("=" * 70)


NO METRICS FOUND

Please:
1. Ensure notebooks 03 and 04 have been executed completely
2. Re-run the extraction cell above
3. Optionally add notebook 02 metrics manually


## Step 2: Model Performance Comparison by Metric

This section creates tables and visualizations for business and technical review:
- One table for all model/dataset/metrics
- Highlight the best models for each metric (recall, f1, ROC-AUC)
- Plot bar charts for top-scoring models per metric


In [27]:
# ============================================================
# Summary Table: All Models, All Metrics
# ============================================================

if len(metrics_df) == 0:
    print('Paste or load your comparison DataFrame in the previous cell!')
else:
    # Map dataset_type to readable names if not present
    if 'dataset' not in metrics_df:
        dataset_name_map = {'raw': 'Raw/Original', 'smote': 'SMOTE Balanced', 'adasyn': 'ADASYN Balanced'}
        metrics_df['dataset'] = metrics_df['dataset_type'].map(dataset_name_map)

    display_cols = [
        'model_name', 'dataset', 'accuracy', 'precision_positive', 'recall_positive', 'f1_positive', 'roc_auc', 'tn', 'fp', 'fn', 'tp'
    ]
    df_display = metrics_df[display_cols].copy()
    # Highlight top for each metric
    highlight = {}
    for m in ['recall_positive', 'f1_positive', 'roc_auc']:
        idx = df_display[m].astype(float).idxmax()
        if idx not in highlight:
            highlight[idx] = m

    def color_max(s):
        color = ['background-color: #b3ffb3' if i in highlight and highlight[i]==s.name else '' for i in s.index]
        return color

    try:
        display(df_display.style.apply(color_max))
    except:
        print(df_display)

    print('\nBest models by metric:')
    for m in ['recall_positive', 'f1_positive', 'roc_auc']:
        best_row = df_display.loc[df_display[m].astype(float).idxmax()]
        print(f"- {m}: {best_row['model_name']} on {best_row['dataset']} (score = {best_row[m]:.4f})")


Paste or load your comparison DataFrame in the previous cell!


In [24]:
# ============================================================
# Bar Plots: Model Comparison by Recall, F1, ROC-AUC
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns

if metrics_df.empty:
    print('No results to plot. Please load or paste your model metrics in the table above.')
else:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    metrics_to_plot = [('recall_positive', 'Recall'), ('f1_positive', 'F1 Score'), ('roc_auc', 'ROC-AUC')]

    for i, (col, title) in enumerate(metrics_to_plot):
        top = metrics_df.groupby(['model_name', 'dataset'])[[col]].max().reset_index()
        sns.barplot(data=top, x='model_name', y=col, hue='dataset', ax=axes[i])
        axes[i].set_title(f'Models Comparison: {title}')
        axes[i].set_ylim(0, 1)
        axes[i].legend(title='Dataset')
    plt.tight_layout()
    plt.show()



No results to plot. Please load or paste your model metrics in the table above.


## Step 3: Confusion Matrix Heatmaps for Top Models

Show confusion matrices for the models with the best recall, F1, and ROC-AUC (one per metric).


In [28]:
# ============================================================
# Confusion Matrix Heatmaps for Top Models (Recall, F1, ROC-AUC)
# ============================================================

if metrics_df.empty:
    print('No confusion matrices to visualize. Please load or paste your model metrics in the table above.')
else:
    def plot_conf_matrix(row, metric_label):
        cm = np.array([[row['tn'], row['fp']], [row['fn'], row['tp']]])
        plt.figure(figsize=(4, 3))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Pred No', 'Pred Yes'],
                    yticklabels=['Actual No', 'Actual Yes'])
        plt.title(f"{row['model_name']} – {row['dataset']}\nConfusion Matrix [{metric_label}]")
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.tight_layout()
        plt.show()

    # Plot for models with max recall, f1, roc-auc
    for metric_label in ['recall_positive', 'f1_positive', 'roc_auc']:
        idx = metrics_df[metric_label].astype(float).idxmax()
        row = metrics_df.loc[idx]
        print(f"\nConfusion Matrix for best {metric_label} model:")
        plot_conf_matrix(row, metric_label)



No confusion matrices to visualize. Please load or paste your model metrics in the table above.


## Step 4: Business-Oriented Model Recommendation

Report best models per metric and the corresponding confusion matrix breakdown for business action:
- Which model catches the most at-risk employees (best recall)?
- What is the trade-off between false alarms and missed cases among the winning models?

📢 **Copy/paste or summarize these conclusions in any business summary deck.**
